## The 2nd day in Davos
### Focus on MCMC methods today  
-  Need to calculate the truncate point

In [5]:
using Distributions
μ = 10
σ = 10
t = 15
s = (t - μ)/σ
d = Normal(0, 1)
i = pdf(d, s)/(1 - cdf(d, s))
meanT = μ + σ*i
varT = σ^2*(1 - i*(i - s))
@printf "mean     = %8.2f  \n" meanT
@printf "var      = %8.2f  \n" varT

mean     =    21.41  
var      =    26.85  


### MC method  
- Much better

In [1]:
using Distributions
μ = 10
σ = 10
z = rand(Normal(μ, σ), 10000)
mc_mean = mean(z[z .> t])
mc_var = var(z[z .> t])
@printf "mean     = %8.2f  \n" mc_mean
@printf "var      = %8.2f  \n" mc_var

LoadError: t not defined
while loading In[1], in expression starting on line 5

### Bivariate case

In [22]:
μ = [0.0
    0.0]

2-element Array{Float64,1}:
 0.0
 0.0

In [23]:
V = [1.0 0.3
      0.3 1.0]

2x2 Array{Float64,2}:
 1.0  0.3
 0.3  1.0

In [24]:
using Distributions
d = MvNormal(μ, V)
XY = rand(d, 10000)'  # remember the t()

10000x2 Array{Float64,2}:
  1.31618     2.14513 
 -0.284121   -0.331597
  0.685744    0.642801
  0.02178     0.288295
  1.06012     0.119728
  1.29534     1.61772 
 -1.02127     0.448311
  0.195229   -1.30403 
  0.766059   -0.269316
 -0.634323   -0.558635
 -0.142248    0.297923
  0.909844   -0.577306
  0.831251   -0.770879
  ⋮                   
 -0.355703    0.911417
 -2.48112    -1.71322 
  2.38353     0.524021
 -0.989931   -1.44586 
 -1.92034     0.61979 
 -1.23565    -1.11905 
  0.0845248   0.888092
  0.0535568  -0.316295
  2.03408     1.42669 
 -1.16051    -0.746082
 -1.03258    -0.463581
 -0.575096   -1.98608 

In [38]:
selY = XY[(XY[:, 1] .> -1) & (XY[:, 1] .< 1), 2]
selY = sort(selY, rev=true)[1:0.05*10000]
length(selY)

500

In [39]:
println("The mean is  ", mean(selY))
println("The var is  ", var(selY))

The mean is  1.840958336989763
The var is  0.16361185881407336


In [49]:
#selX = XY[(XY[:, 2] .> -1) & (XY[:, 2] .< 1), 1]
selX = XY[:, 1]    # not select the second trait     
selX = sort(selX, rev=true)[1:0.05*10000]
println(mean(selX))
println(var(selX))

2.091132480497743
0.12196491867784594


### MCMC

In [56]:
using Distributions
function SimData(nanimal, nloci, bme, bsd, resd)
    X = sample(0:2, (nanimal, nloci))
    srand(123) # set seed
    b = rand(Normal(bme, bsd), size(X, 2))
    e = rand(Normal(0, resd), size(X, 1))
    y = X*b + e
    return(y, X, b, e)     # tuple returned
end

SimData (generic function with 1 method)

In [100]:
using Distributions
# set the parameters
ninter = 2000
nanimal = 6
nloci = 4
bme = 0
bsd = 2.5
resd = 1.0
getdata = SimData(nanimal, nloci, bme, bsd, resd)
y = getdata[1]
X = getdata[2]
# for store mu and b
bSample = Array(Float64, ninter, nloci)
muSample = Array(Float64, ninter)
# initiate the mu and b
b = fill(0.01, nloci)
ONE = fill(1.0, nanimal)
mu = 0.0
# start the Gibbs sampler
for (i in 1:ninter)
    if i%100 == 0
        println("The round is $i")
    end
    # sampling mu from a Normal distribution
    #mu = rand(Normal(((ONE'y - ONE'X*b)/nanimal)[1], sqrt(resd^2/nanimal)))[1]
    mu = rand(Normal(((ONE'y - ONE'X*b)/nanimal)[1]))
    # sampling b from a Normal distribution
    z = fill(0.0, nanimal)
    for (mk in 1:nloci)
        btemp = copy(b)
        btemp[mk] = 0.0
        z = X[:, mk]
        mm = (z'y - z'X*btemp - z'ONE*mu)/(z'z)  # the mean of the Normal distribution
        b[mk] = rand(Normal(mm[1], sqrt(resd^2/(z'z)[1])))
        #b[mk] = rand(Normal(mm[1], sqrt(resd^2)))
    end
    # store the samplers
    bSample[i, :] = b
    muSample[i] = mu
end
# posterior mean     
bsol = mean(bSample, 1)
GEBV = X*bsol'





The round is 100
The round is 200
The round is 300
The round is 400
The round is 500
The round is 600
The round is 700
The round is 800
The round is 900
The round is 1000
The round is 1100
The round is 1200
The round is 1300
The round is 1400
The round is 1500
The round is 1600
The round is 1700
The round is 1800
The round is 1900
The round is 2000


6x1 Array{Float64,2}:
 13.5258 
  9.82991
 16.9011 
 17.1539 
  9.65904
 10.9974 

In [92]:
resd^2/(z'z)[1]        
#rand(Normal(mm, sqrt(resd^2/(z'z))[1]))


0.07692307692307693

In [66]:
rand(Normal(8.99685))

7.802866226882911